In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("nlp").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/02 19:27:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection", inferSchema = True, sep = "\t")

In [4]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [5]:
data = data.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")

In [6]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import length

In [9]:
data = data.withColumn("length", length(data["text"]))

In [10]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [11]:
data.groupBy("class").mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## NLP features: tokenization, removing stop words, count vertorizer, inverse document frequency, string indexer

In [12]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [13]:
tokenizer = Tokenizer(inputCol = "text", outputCol = "token_text")

In [14]:
stop_remove = StopWordsRemover(inputCol = "token_text", outputCol = "stop_token")

In [15]:
count_vec = CountVectorizer(inputCol = "stop_token", outputCol = "c_vec")

## ft_idf: term frequency - inverse document frequency

In [16]:
idf = IDF(inputCol = "c_vec", outputCol = "tf_idf") # tf_idf = term frequency-inverse document frequency 

In [17]:
ham_spam_to_numeric = StringIndexer(inputCol = "class", outputCol = "label")

In [18]:
from pyspark.ml.feature import VectorAssembler

In [19]:
clean_up = VectorAssembler(inputCols = ["tf_idf", "length"], outputCol = "features")

In [20]:
from pyspark.ml.classification import NaiveBayes

In [21]:
nb = NaiveBayes()

In [22]:
from pyspark.ml import Pipeline

In [23]:
data_prep_pipe = Pipeline(stages = [ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [25]:
cleaner = data_prep_pipe.fit(data)

In [26]:
clean_data = cleaner.transform(data)

In [27]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [28]:
clean_data = clean_data.select("label", "features")

In [29]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [30]:
training, test = clean_data.randomSplit([0.7, 0.3])

In [32]:
spam_detector = nb.fit(training)

22/05/02 20:25:03 WARN DAGScheduler: Broadcasting large task binary with size 1142.7 KiB
22/05/02 20:25:05 WARN DAGScheduler: Broadcasting large task binary with size 1126.2 KiB


In [33]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [34]:
test_results = spam_detector.transform(test)

In [35]:
test_results.show()

22/05/02 20:26:43 WARN DAGScheduler: Broadcasting large task binary with size 1360.1 KiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-788.68382904527...|[1.0,6.5078524296...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1061.1394874915...|[1.0,4.7735032839...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-1000.7034200312...|[1.0,7.7158888033...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-816.96740002769...|[1.0,1.7709207031...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-662.45344021506...|[1.0,3.5301881353...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-541.63076972174...|[1.0,1.9847693190...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-871.55777104481...|[1.0,4.9713498422...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-217.28348422823...|[1.0,6.2012284569...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-686.08435008339...|[1.0,1.5036784480...|       0.0|
|  0.0|(13424,[0

In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [37]:
acc_eval = MulticlassClassificationEvaluator()

In [39]:
acc = acc_eval.evaluate(test_results)

22/05/02 20:29:22 WARN DAGScheduler: Broadcasting large task binary with size 1365.2 KiB


In [42]:
print("Accuracy of NaiveBayes Model")
print(acc)

Accuracy of NaiveBayes Model
0.9202912788619935
